In [ ]:
%%configure -f 
{ "jars": ["wasb:///powerbi/jars/adal4j-1.1.2.jar", "wasb:///powerbi/microsoft-spark-powerbi-connector_2.10-0.6.0.jar"] }

In [ ]:
case class Circle(name: String, radius: Double)

In [ ]:
import scala.collection.mutable.ListBuffer

val randomGenerator = scala.util.Random

val circleListBuffer: ListBuffer[Circle] = new ListBuffer

for (i <- 1 to 5) {

    circleListBuffer += new Circle(f"Circle$i", randomGenerator.nextInt(1000))
}

In [ ]:
val circleRDD = sc.parallelize(circleListBuffer)

circleRDD.count

In [ ]:
import scala.math
import org.apache.spark.sql.functions._

val computeCircleArea = (circleRadius: Double) => {
    
    math.Pi * circleRadius * circleRadius
}

In [ ]:
case class CircleAreaByRadius(radius: Double, area: Double)

In [ ]:
val circleAreaByRadiusRDD = circleRDD.map(x => new CircleAreaByRadius(x.radius, computeCircleArea(x.radius)))

circleAreaByRadiusRDD.count

In [ ]:
import com.microsoft.spark.powerbi.authentication._
import com.microsoft.spark.powerbi.models._
import com.microsoft.spark.powerbi.common._

val powerbiClientId = "***Enter PowerBI Client Id here***" 
val powerbiAccountUsername = "***Enter PowerBI Account Username here***"
val powerbiAccountPassword = "***Enter PowerBI Account Password here***"

def initializeAuthentication(): PowerBIAuthentication = {

    val powerBIAuthentication: PowerBIAuthentication = new PowerBIAuthentication(
      PowerBIURLs.Authority,
      PowerBIURLs.Resource,
      powerbiClientId,
      powerbiAccountUsername,
      powerbiAccountPassword
    )

    powerBIAuthentication
}

In [ ]:
//Initialize PowerBI Authentication

val powerbiAuthentication: PowerBIAuthentication = initializeAuthentication()

//Create PowerBI dataset and table

val powerbiTableName = "CircleRadiusAreaRDDTable"
val powerbiDatasetName = "CircleRDDDataset"

val circleTableColumns = Map("radius" -> PowerBIDataTypes.Double.toString(), "area" -> PowerBIDataTypes.Double.toString())

val circleTable = PowerBIUtils.defineTable(powerbiTableName, circleTableColumns)

val powerbiTableList = List[table](circleTable)

val powerbiDatasetDetails = PowerBIUtils.getOrCreateDataset(powerbiDatasetName, powerbiTableList,
PowerBIOptions.basicFIFO, powerbiAuthentication.getAccessToken())

println(powerbiDatasetDetails)

In [ ]:
import com.microsoft.spark.powerbi.extensions.RDDExtensions._

circleAreaByRadiusRDD.toPowerBI(powerbiDatasetDetails, powerbiTableList.head, powerbiAuthentication)